# SQL Injection Detenction

## Load Libraries

In [1]:
import re
import joblib
import numpy as np
import pandas as pd
from texttable import Texttable
import matplotlib.pyplot as plt
from scipy.sparse import hstack
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier, StackingClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

## Load Dataset

In [2]:
training = pd.read_csv('./Dataset/trainingdata.csv')
training.drop_duplicates(inplace=True)

testing = pd.read_csv('./Dataset/testingdata.csv')
testing.drop_duplicates(inplace=True)

data = pd.concat([training, testing], axis=0)
data.drop_duplicates(inplace=True)
data.reset_index(drop=True, inplace=True)
data = data.dropna(subset=['Label', 'Query'])

## Functions definition

In [3]:
# Funzione per estrarre le caratteristiche da una query SQL
def extract_features(query):
    features = {}
    features['no_sngle_quts'] = query.count("'")  # Number of single quotes
    features['no_dble_quts'] = query.count('"')  # Number of double quotes
    features['no_punctn'] = sum([1 for char in query if char in '!@#$%^&*()-_=+[{]};:\'",<.>/?\\|`~'])  # Number of punctuations
    features['no_sgle_cmnt'] = query.count('--')  # Number of single line comments
    features['no_mlt_cmnt'] = query.count('/*') + query.count('*/')  # Number of multi-line comments
    features['no_whte_spce'] = query.count(' ')  # Number of white spaces
    features['no_nrml_kywrds'] = len(re.findall(r'\b(select|from|where|insert|delete|update|join|union)\b', query, re.IGNORECASE))  # Normal keywords
    features['no_hmfl_kywrds'] = len(re.findall(r'\b(exec|shutdown|cmdshell|ascii|hex|char|concat)\b', query, re.IGNORECASE))  # Harmful keywords
    features['no_prctge'] = query.count('%')  # Number of percentage symbols
    features['no_log_oprtr'] = len(re.findall(r'\b(and|or|not)\b', query, re.IGNORECASE))  # Logical operators
    features['no_oprtr'] = sum([1 for char in query if char in '=<>'])  # Number of operators
    features['no_null_valus'] = query.lower().count('null')  # Number of null values
    features['no_hexdcml_valus'] = len(re.findall(r'0x[0-9a-fA-F]+', query))  # Number of hexadecimal values
    features['no_db_info_cmnds'] = len(re.findall(r'\b(database|information_schema|version)\b', query, re.IGNORECASE))  # Database information commands
    features['no_roles'] = len(re.findall(r'\b(admin|user|guest)\b', query, re.IGNORECASE))  # Roles
    features['no_ntwr_cmnds'] = len(re.findall(r'\b(load_file|benchmark|sleep)\b', query, re.IGNORECASE))  # Network commands
    features['no_lanage_cmnds'] = len(re.findall(r'\b(exec|declare|open|fetch|close|deallocate|prepare|execute)\b', query, re.IGNORECASE))  # Language commands
    features['no_alphabet'] = len(re.findall(r'[a-zA-Z]', query))  # Number of alphabets
    features['no_digits'] = len(re.findall(r'\d', query))  # Number of digits
    features['no_spl_chrtr'] = len(re.findall(r'[^a-zA-Z0-9\s]', query))  # Number of special characters
    return features

# Evaluation function
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1, "conf_matrix": conf_matrix, "roc_auc": roc_auc}

# Function to plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, title, filename):
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap=plt.cm.Blues)
    plt.title(title, fontsize=16)
    plt.xlabel('Predicted Label', fontsize=14)
    plt.ylabel('True Label', fontsize=14)
    
    # Increase font size of the numbers in the confusion matrix
    for labels in disp.text_.ravel():
        labels.set_fontsize(14)
    
    plt.savefig(filename, format='png', dpi=300, bbox_inches='tight')
    plt.close()

# Function to plot ROC-AUC Curve
def plot_roc_auc_curve(y_true, y_pred_prob, model_name, filename):
    fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
    roc_auc = auc(fpr, tpr)
    
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC-AUC Curve: {model_name}')
    plt.legend(loc='lower right')
    plt.savefig(filename, format='png', dpi=300, bbox_inches='tight')
    plt.close()

def plot_classification_report(models_stats, filename_prefix):
    metrics = ['roc_auc', 'precision', 'recall', 'f1']
    models = list(models_stats.keys())
    
    # Prepare the data
    report_data = []
    for model, stats in models_stats.items():
        report_data.append([stats['roc_auc'], stats['precision'], stats['recall'], stats['f1']])
    
    # Convert to DataFrame for easier plotting
    report_df = pd.DataFrame(report_data, columns=metrics, index=models)

    colors = ['#007bff', '#28a745', '#dc3545', '#ffc107']

    # Plot all models in one go
    ax = report_df.plot(kind='bar', figsize=(14, 8), width=0.7, color=colors, legend=False)  # Adjusted figure size
    plt.title("Classification Report", fontsize=16)
    plt.xticks(rotation=45, ha='right')  # Rotate labels for better readability
    plt.ylabel("Score (%)", fontsize=14)
    plt.xticks(fontsize=14)  # Increase font size for x-axis labels
    plt.yticks(fontsize=14)  # Increase font size for y-axis labels
    
    # Set y-axis range for better differentiation
    ax.set_ylim(0.8, 1.05)  # Adjust this range as per your needs
    
    # Annotate the bars with the actual values
    for p in ax.patches:
        ax.annotate(f'{p.get_height():.4f}', 
                    (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha='center', va='baseline', 
                    xytext=(0, -100), 
                    textcoords='offset points', fontsize=13, rotation=90)
    
    # Move legend to bottom left corner
    plt.legend(loc='lower left')
    plt.tight_layout()
    plt.savefig(f"{filename_prefix}_all_models.png", format='png', dpi=300)
    plt.close()

## Feature extraction

In [4]:
# Estrazione le caratteristiche dalle query
data_features = data['Query'].apply(extract_features)
data_features_df = pd.DataFrame(data_features.tolist())

# Combina le caratteristiche estratte con le etichette
df = pd.concat([data_features_df, data['Label']], axis=1)

# Separazione delle caratteristiche (X) e le etichette (y)
X_numerical = df.drop(columns=['Label'])
y = df['Label']

# Salva la colonna 'Query' per la successiva creazione delle BoW
X_text = data['Query']

# Divide i dati in set di addestramento e test mantenendo la colonna 'Query'
X_train_numerical, X_test_numerical, X_train_text, X_test_text, y_train, y_test = train_test_split(
    X_numerical, X_text, y, test_size=0.4)

# Trasforma le caratteristiche testuali in TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=1000)
X_train_query = vectorizer.fit_transform(X_train_text)
X_test_query = vectorizer.transform(X_test_text)

# Combina le caratteristiche numeriche e testuali
X_train = hstack((X_train_numerical.values, X_train_query)).tocsr()
X_test = hstack((X_test_numerical.values, X_test_query)).tocsr()

## Evaluate of models

### Gradient Boosting Machine

In [5]:
gbm = GradientBoostingClassifier()
gbm.fit(X_train, y_train)
y_pred_gbm = gbm.predict(X_test)

### AdaBoost

In [6]:
ada = AdaBoostClassifier()
ada.fit(X_train, y_train)
y_pred_ada = ada.predict(X_test)

c:\Users\hp\OneDrive - uniroma1.it\Desktop\Uni\3°Anno uni\Sicurezza\SQLi detenction\venv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


### XGBoost

In [7]:
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

c:\Users\hp\OneDrive - uniroma1.it\Desktop\Uni\3°Anno uni\Sicurezza\SQLi detenction\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [17:12:02] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


### LightGBM

In [8]:
lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train)
y_pred_lgbm = lgbm.predict(X_test)

[LightGBM] [Info] Number of positive: 40487, number of negative: 33220
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.220572 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 221934
[LightGBM] [Info] Number of data points in the train set: 73707, number of used features: 1020
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.549297 -> initscore=0.197829
[LightGBM] [Info] Start training from score 0.197829


### Logistic Regression

In [9]:
log_reg = LogisticRegression(max_iter=100000)
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.fit(X_train, y_train).predict(X_test)

#kf = KFold(n_splits=5, shuffle=True)
#results = cross_validate(log_reg, X_train, y_train, cv=kf, scoring=['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted'])
#print(f"Mean Accuracy: {results['test_accuracy'].mean():.2f}")
#print(f"Mean Precision: {results['test_precision_weighted'].mean():.2f}")
#print(f"Mean Recall: {results['test_recall_weighted'].mean():.2f}")
#print(f"Mean F1 Score: {results['test_f1_weighted'].mean():.2f}")

### Random Forest

In [10]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

### K-Nearest Neighbors

In [11]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

### Decision Tree

In [12]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

### Stacking Classifier

In [13]:
estimators = [
    ('gbm', gbm),
    ('ada', ada),
    ('xgb', xgb),
    ('lgbm', lgbm),
    ('rf', rf),
    ('log_reg', log_reg),
    ('knn', knn),
    ('dt', dt)
]

stacking = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
stacking.fit(X_train.astype(np.float32), y_train.astype(np.float32))
y_pred_stacking = stacking.predict(X_test.astype(np.float32))

c:\Users\hp\OneDrive - uniroma1.it\Desktop\Uni\3°Anno uni\Sicurezza\SQLi detenction\venv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\hp\OneDrive - uniroma1.it\Desktop\Uni\3°Anno uni\Sicurezza\SQLi detenction\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [17:27:46] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Number of positive: 40487, number of negative: 33220
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.196431 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 221932
[LightGBM] [Info] Number of data points in the train set: 73707, number of used features: 1020
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.549297 -> initscore=0.197829
[LightGBM] [Info] Start training from score 0.197829


c:\Users\hp\OneDrive - uniroma1.it\Desktop\Uni\3°Anno uni\Sicurezza\SQLi detenction\venv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\hp\OneDrive - uniroma1.it\Desktop\Uni\3°Anno uni\Sicurezza\SQLi detenction\venv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\hp\OneDrive - uniroma1.it\Desktop\Uni\3°Anno uni\Sicurezza\SQLi detenction\venv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\hp\OneDrive - uniroma1.it\Desktop\Uni\3°Anno uni\Sicurezza

[LightGBM] [Info] Number of positive: 32389, number of negative: 26576
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.153521 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 199298
[LightGBM] [Info] Number of data points in the train set: 58965, number of used features: 1020
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.549292 -> initscore=0.197810
[LightGBM] [Info] Start training from score 0.197810
[LightGBM] [Info] Number of positive: 32389, number of negative: 26576
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.185523 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 201525
[LightGBM] [Info] Number of data points in the train set: 58965, number of used features: 1020
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.549292 -> initscore=0.1978

## Graphs print

In [14]:
# Stampa delle confusion matrix
plot_confusion_matrix(y_test, y_pred_gbm, "Gradient Boosting Machine Confusion Matrix", "./images_Bow/confusion_matrix/gbm_confusion_matrix.png")
plot_confusion_matrix(y_test, y_pred_ada, "AdaBoost Confusion Matrix", "./images_Bow/confusion_matrix/ada_confusion_matrix.png")
plot_confusion_matrix(y_test, y_pred_xgb, "XGBoost Confusion Matrix", "./images_Bow/confusion_matrix/xgb_confusion_matrix.png")
plot_confusion_matrix(y_test, y_pred_lgbm, "LightGBM Confusion Matrix", "./images_Bow/confusion_matrix/lgbm_confusion_matrix.png")
plot_confusion_matrix(y_test, y_pred_log_reg, "Logistic Regression Confusion Matrix", "./images_Bow/confusion_matrix/logreg_confusion_matrix.png")
plot_confusion_matrix(y_test, y_pred_rf, "Random Forest Confusion Matrix", "./images_Bow/confusion_matrix/rf_confusion_matrix.png")
plot_confusion_matrix(y_test, y_pred_knn, "K-Nearest Neighbors Confusion Matrix", "./images_Bow/confusion_matrix/knn_confusion_matrix.png")
plot_confusion_matrix(y_test, y_pred_dt, "Decision Tree Confusion Matrix", "./images_Bow/confusion_matrix/dt_confusion_matrix.png")
plot_confusion_matrix(y_test, y_pred_stacking, "Stacking Classifier Confusion Matrix", "./images_Bow/confusion_matrix/stacking_confusion_matrix.png")

# Stampa delle ROC-AUC Curve
plot_roc_auc_curve(y_test, y_pred_gbm, "Gradient Boosting Machine", "./images_Bow/roc_auc/gbm_roc_auc.png")
plot_roc_auc_curve(y_test, y_pred_ada, "AdaBoost", "./images_Bow/roc_auc/ada_roc_auc.png")
plot_roc_auc_curve(y_test, y_pred_xgb, "XGBoost", "./images_Bow/roc_auc/xgb_roc_auc.png")
plot_roc_auc_curve(y_test, y_pred_lgbm, "LightGBM", "./images_Bow/roc_auc/lgbm_roc_auc.png")
plot_roc_auc_curve(y_test, y_pred_log_reg, "Logistic Regression", "./images_Bow/roc_auc/logreg_roc_auc.png")
plot_roc_auc_curve(y_test, y_pred_rf, "Random Forest", "./images_Bow/roc_auc/rf_roc_auc.png")
plot_roc_auc_curve(y_test, y_pred_knn, "K-Nearest Neighbors", "./images_Bow/roc_auc/knn_roc_auc.png")
plot_roc_auc_curve(y_test, y_pred_dt, "Decision Tree", "./images_Bow/roc_auc/dt_roc_auc.png")
plot_roc_auc_curve(y_test, y_pred_stacking, "Stacking Classifier", "./images_Bow/roc_auc/stacking_roc_auc.png")

In [15]:
# Raccoglie le statistiche per ogni modello, accuratazza, precisione, richiamo, f1, matrice di confusione e ROC-AUC
models_stats = {
    "Gradient Boosting Machine": evaluate_model(y_test, y_pred_gbm),
    "AdaBoost": evaluate_model(y_test, y_pred_ada),
    "XGBoost": evaluate_model(y_test, y_pred_xgb),
    "LightGBM": evaluate_model(y_test, y_pred_lgbm),
    "Logistic Regression": evaluate_model(y_test, y_pred_log_reg),
    "Random Forest": evaluate_model(y_test, y_pred_rf),
    "K-Nearest Neighbors": evaluate_model(y_test, y_pred_knn),
    "Decision Tree": evaluate_model(y_test, y_pred_dt),
    "Stacking Classifier": evaluate_model(y_test, y_pred_stacking)
}

# Creazione di un report di classificazione
plot_classification_report(models_stats, "./images_Bow/classification/classification_report")

In [16]:
# Create a Texttable object
table = Texttable()

# Add headers
table.header(["Model", "Accuracy", "Precision", "Recall", "F1 Score", "ROC AUC"])

# Add rows for each model's statistics
for model, stats in models_stats.items():
    table.add_row([model, stats['accuracy'], stats['precision'], stats['recall'], stats['f1'], stats['roc_auc']])

# Draw the table
print(table.draw())

+-------------------------+----------+-----------+--------+----------+---------+
|          Model          | Accuracy | Precision | Recall | F1 Score | ROC AUC |
+=========================+==========+===========+========+==========+=========+
| Gradient Boosting       | 0.991    | 0.993     | 0.991  | 0.992    | 0.991   |
| Machine                 |          |           |        |          |         |
+-------------------------+----------+-----------+--------+----------+---------+
| AdaBoost                | 0.989    | 0.990     | 0.989  | 0.990    | 0.989   |
+-------------------------+----------+-----------+--------+----------+---------+
| XGBoost                 | 0.996    | 0.998     | 0.996  | 0.997    | 0.996   |
+-------------------------+----------+-----------+--------+----------+---------+
| LightGBM                | 0.997    | 0.998     | 0.996  | 0.997    | 0.997   |
+-------------------------+----------+-----------+--------+----------+---------+
| Logistic Regression     | 

## Save models 

In [17]:
# Save each model to a file
joblib.dump(vectorizer, './models/BoW/vectorizer.pkl')
joblib.dump(gbm, './models/BoW/gbm_model.pkl')
joblib.dump(ada, './models/BoW/ada_model.pkl')
joblib.dump(xgb, './models/BoW/xgb_model.pkl')
joblib.dump(lgbm, './models/BoW/lgbm_model.pkl')
joblib.dump(log_reg, './models/BoW/log_reg_model.pkl')
joblib.dump(rf, './models/BoW/rf_model.pkl')
joblib.dump(knn, './models/BoW/knn_model.pkl')
joblib.dump(dt, './models/BoW/dt_model.pkl')
joblib.dump(stacking, './models/BoW/stacking_model.pkl')

print("Models have been saved successfully.")

Models have been saved successfully.
